# FIAP BANK

Desenvolvimento de um sistema para desenvolver habilidades em herança e polimorfismo.

**O FIAP Bank** simula um banco no qual é possível emular as seguintes tarefas:

**CRUD de usuários**

1.   Gerentes
2.   Correntistas

Ambos carregam os dados de login e senha.

**Contas**
1.   Abrir
2.   Fechar

**Transações**

1.   Transferências
2.   Depósitos
3.   Saques



**Permissões**

1.   Somente gerentes conseguem visualizar todas as contas.
2.   Correntistas só podem ver a própria conta.


Importante: Não usamos nenhum banco de dados para armazenar as transações.
Todo o dicionário está armazenado em um arquivo JSON.

In [ ]:
import json
from abc import ABC, abstractmethod

#Importação para administração da senha
from getpass import getpass

# Classes de Usuários, Correntistas e Gerentes

In [ ]:
class Usuario(ABC):
    def __init__(self, usuario, senha):
        self.usuario = usuario
        self.senha = senha

    @abstractmethod
    def tipo(self):
        pass

class Correntista(Usuario):
    def __init__(self, usuario, senha):
        super().__init__(usuario, senha)
        self.saldo = 0.0
        self.transacoes = []

    def tipo(self):
        return "Correntista"

class Gerente(Usuario):
    def __init__(self, usuario, senha):
        super().__init__(usuario, senha)

    def tipo(self):
        return "Gerente"


# Classes do banco para gerenciar usuários e transações

In [ ]:
class Banco:
    def __init__(self, arquivo_dados):
        self.arquivo_dados = arquivo_dados
        self.usuarios = self.carregar_dados()

    def carregar_dados(self):
        try:
            with open(self.arquivo_dados, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            return {"correntistas": {}, "gerentes": {}}

    def salvar_dados(self):
        with open(self.arquivo_dados, 'w') as f:
            json.dump(self.usuarios, f, indent=4)

    def cadastrar_correntista(self, usuario, senha):
        if usuario in self.usuarios["correntistas"]:
            print("Usuário já existe!")
        else:
            self.usuarios["correntistas"][usuario] = {
                "senha": senha, "saldo": 0.0, "transacoes": []}
            self.salvar_dados()
            print(f"Correntista {usuario} cadastrado com sucesso!")

    def cadastrar_gerente(self, usuario, senha):
        if usuario in self.usuarios["gerentes"]:
            print("Usuário já existe!")
        else:
            self.usuarios["gerentes"][usuario] = {"senha": senha}
            self.salvar_dados()
            print(f"Gerente {usuario} cadastrado com sucesso!")

    def autenticar(self, usuario, senha, tipo):
        if tipo == "Correntista":
            return usuario in self.usuarios["correntistas"] and self.usuarios["correntistas"][usuario]["senha"] == senha
        elif tipo == "Gerente":
            return usuario in self.usuarios["gerentes"] and self.usuarios["gerentes"][usuario]["senha"] == senha
        return False

    def verificar_saldo(self, usuario):
        return self.usuarios["correntistas"][usuario]["saldo"]

    def depositar(self, usuario, valor):
        self.usuarios["correntistas"][usuario]["saldo"] += valor
        self.usuarios["correntistas"][usuario]["transacoes"].append(f"Depósito de {valor}")
        self.salvar_dados()

    def sacar(self, usuario, valor):
        if self.usuarios["correntistas"][usuario]["saldo"] >= valor:
            self.usuarios["correntistas"][usuario]["saldo"] -= valor
            self.usuarios["correntistas"][usuario]["transacoes"].append(f"Saque de {valor}")
            self.salvar_dados()
        else:
            print("Saldo insuficiente.")

    def transferir(self, origem, destino, valor):
        if origem in self.usuarios["correntistas"] and destino in self.usuarios["correntistas"]:
            if self.usuarios["correntistas"][origem]["saldo"] >= valor:
                self.usuarios["correntistas"][origem]["saldo"] -= valor
                self.usuarios["correntistas"][destino]["saldo"] += valor
                self.usuarios["correntistas"][origem]["transacoes"].append(f"Transferência de {valor} para {destino}")
                self.usuarios["correntistas"][destino]["transacoes"].append(f"Transferência de {valor} de {origem}")
                self.salvar_dados()
            else:
                print("Saldo insuficiente.")
        else:
            print("Conta de origem ou destino não encontrada.")

    def encerrar_conta(self, usuario):
        if usuario in self.usuarios["correntistas"]:
            del self.usuarios["correntistas"][usuario]
            self.salvar_dados()
            print(f"Conta do correntista {usuario} encerrada com sucesso.")
        else:
            print("Usuário não encontrado.")

    def listar_saldos_e_transacoes(self):
        for usuario, dados in self.usuarios["correntistas"].items():
            print(f"Usuário: {usuario}, Saldo: {dados['saldo']}")
            for transacao in dados["transacoes"]:
                print(f"  - {transacao}")


In [ ]:
def main():
    banco = Banco('banco_dados.json')

    while True:
        print("1. Cadastrar Correntista")
        print("2. Cadastrar Gerente")
        print("3. Login Correntista")
        print("4. Login Gerente")
        print("5. Sair")
        opcao = input("Escolha uma opção: ")

        if opcao == "1":
            usuario = input("Nome de usuário: ")
            senha = getpass("Senha: ")
            banco.cadastrar_correntista(usuario, senha)
        elif opcao == "2":
            usuario = input("Nome de usuário: ")
            senha = getpass("Senha: ")
            banco.cadastrar_gerente(usuario, senha)
        elif opcao == "3":
            usuario = input("Nome de usuário: ")
            senha = getpass("Senha: ")
            if banco.autenticar(usuario, senha, "Correntista"):
                while True:
                    print(f"Bem-vindo, {usuario}")
                    print("1. Verificar Saldo")
                    print("2. Depositar")
                    print("3. Sacar")
                    print("4. Transferir")
                    print("5. Encerrar Conta")
                    print("6. Logout")
                    opcao_correntista = input("Escolha uma opção: ")
                    if opcao_correntista == "1":
                        print(f"Seu saldo é: {banco.verificar_saldo(usuario)}")
                    elif opcao_correntista == "2":
                        valor = float(input("Valor a depositar: "))
                        banco.depositar(usuario, valor)
                    elif opcao_correntista == "3":
                        valor = float(input("Valor a sacar: "))
                        banco.sacar(usuario, valor)
                    elif opcao_correntista == "4":
                        destino = input("Transferir para (usuário): ")
                        valor = float(input("Valor a transferir: "))
                        banco.transferir(usuario, destino, valor)
                    elif opcao_correntista == "5":
                        confirmacao = input("Tem certeza que deseja encerrar a conta? (s/n): ")
                        if confirmacao.lower() == 's':
                            banco.encerrar_conta(usuario)
                            break
                    elif opcao_correntista == "6":
                        break
            else:
                print("Usuário ou senha incorretos.")
        elif opcao == "4":
            usuario = input("Nome de usuário: ")
            senha = getpass("Senha: ")
            if banco.autenticar(usuario, senha, "Gerente"):
                while True:
                    print(f"Bem-vindo, {usuario} (Gerente)")
                    print("1. Listar Saldos e Transações")
                    print("2. Logout")
                    opcao_gerente = input("Escolha uma opção: ")
                    if opcao_gerente == "1":
                        banco.listar_saldos_e_transacoes()
                    elif opcao_gerente == "2":
                        break
            else:
                print("Usuário ou senha incorretos.")
        elif opcao == "5":
            break

if __name__ == "__main__":
    main()


fatal: not a git repository (or any of the parent directories): .git
